In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.notebook import tqdm
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import ElectraModel, ElectraTokenizerFast
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_cosine_schedule_with_warmup


tokenizer = ElectraTokenizerFast.from_pretrained("kykim/electra-kor-base")

In [ ]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/project/'
cate = '성별'
df = pd.read_csv(PATH + f'{cate}.csv')

In [ ]:
df = df[['문장', f'{cate}']]

In [ ]:
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch import torch
from sklearn.model_selection import train_test_split

def dataSplit(dataset, y_label):
  X_train, X_val= train_test_split(dataset, test_size = 0.2, stratify = dataset[y_label], random_state =427)
  return X_train, X_val

In [ ]:
X_train, X_test = dataSplit(df, cate)

# validation 추가
X_train, X_val = dataSplit(X_train, cate)

In [ ]:
class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk

    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        # target이 없는경우 (즉, 문장만 입력된 경우)
        if len(row) <= 1:
            text = row[0]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask     
            
        # target이 있는 경우 (원래 코드)
        else:
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask, y

In [ ]:
train_set = LoadDataset(X_train, tokenizer)
val_set = LoadDataset(X_val, tokenizer)
test_set = LoadDataset(X_test, tokenizer)

In [ ]:
model = ElectraForSequenceClassification.from_pretrained('kykim/electra-kor-base')

Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at kykim/electra-kor-base and are newly initialized: ['classifier.out_proj.weight', 'clas

In [ ]:
model.classifier.out_proj =  nn.Sequential( nn.Linear(768, 1),
                                           nn.Sigmoid() )

# model.classifier.out_proj = nn.Linear(256, 1) # model 변경

In [ ]:
device = torch.device("cuda:0")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
epochs = 300 # epochs 증가
batch_size = 32
warmup_ratio=0.1
t_total = len(train_set) * epochs
optimizer = AdamW(model.parameters(), lr=1e-5, eps = 1e-8) # lr 1/10으로 변경
train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size) # val loader 추가
test_loader = DataLoader(test_set, batch_size = batch_size) # test loader 추가
loss_f = nn.BCEWithLogitsLoss() # loss f 변경

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

In [ ]:
# https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path=f'/content/gdrive/MyDrive/checkpoint_{cate}.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
early_stopping = EarlyStopping(patience = 7, verbose = True)

In [ ]:
from sklearn.metrics import roc_auc_score

for i in tqdm(range(epochs)):
    train_loss_list = [] # 변수 변경
    val_loss_list = []
    val_score_list = []

    epoch_train_loss = []
    epoch_val_loss = []
    epoch_val_score = []
    # train
    model.train()
    for input_ids_batch, attention_masks_batch, y_batch in train_loader:
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
        loss = loss_f(y_pred.type(torch.FloatTensor), y_batch.type(torch.FloatTensor))

        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss_list.append(loss.item())

    # validation loss
    model.eval()
    for input_ids_batch_val, attention_masks_batch_val, y_batch_val in val_loader:
        input_ids_batch_val = input_ids_batch_val.to(device)
        attention_masks_batch_val = attention_masks_batch_val.to(device)
        y_batch_val = y_batch_val.to(device)
        y_pred_val = model(input_ids_batch_val, attention_mask = attention_masks_batch_val).logits.reshape(-1)
        loss = loss_f(y_pred_val.type(torch.FloatTensor), y_batch_val.type(torch.FloatTensor))
        val_score = roc_auc_score(y_batch_val.tolist(), y_pred_val.tolist())
        val_loss_list.append(loss.item())
        val_score_list.append(val_score)

    # epoch당 loss 계산 (for early stopping)
    train_loss = np.average(train_loss_list)
    val_loss = np.average(val_loss_list)
    val_score = np.average(val_score_list)

    epoch_train_loss.append(train_loss)
    epoch_val_loss.append(val_loss)
    epoch_val_score.append(val_score)
    epoch_len = len(str(epochs))

    print_msg = (f'[{i:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {val_loss:.5f} ' +
                 f'valid_score: {val_score:.5f}')

    print(print_msg)
    
    # clear lists to track next epoch
    train_loss_list = []
    val_loss_list = []
    val_score_list = []
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print('early stopping')
        break
    
model.load_state_dict(torch.load(f'/content/gdrive/MyDrive/checkpoint_{cate}.pt', map_location=device))

  0%|          | 0/300 [00:00<?, ?it/s]

[  0/300] train_loss: 0.69468 valid_loss: 0.65087 valid_score: 0.82331
Validation loss decreased (inf --> 0.650871).  Saving model ...
[  1/300] train_loss: 0.59539 valid_loss: 0.55226 valid_score: 0.94316
Validation loss decreased (0.650871 --> 0.552261).  Saving model ...
[  2/300] train_loss: 0.55268 valid_loss: 0.54622 valid_score: 0.95376
Validation loss decreased (0.552261 --> 0.546225).  Saving model ...
[  3/300] train_loss: 0.54523 valid_loss: 0.54628 valid_score: 0.94974
EarlyStopping counter: 1 out of 7
[  4/300] train_loss: 0.53896 valid_loss: 0.54229 valid_score: 0.95647
Validation loss decreased (0.546225 --> 0.542290).  Saving model ...
[  5/300] train_loss: 0.53254 valid_loss: 0.54134 valid_score: 0.95397
Validation loss decreased (0.542290 --> 0.541340).  Saving model ...
[  6/300] train_loss: 0.53226 valid_loss: 0.54211 valid_score: 0.94861
EarlyStopping counter: 1 out of 7
[  7/300] train_loss: 0.53153 valid_loss: 0.54408 valid_score: 0.96136
EarlyStopping counter: 2

<All keys matched successfully>

In [ ]:
model.eval()
score_list = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    input_ids_batch = input_ids_batch.to(device)
    attention_masks_batch = attention_masks_batch.to(device)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#    print(y_pred)
    try:
        score = roc_auc_score(y_batch.tolist(), y_pred.tolist())
        score_list.append(score)
    except: pass


print("epoch roc_auc:", np.mean(score_list))

  0%|          | 0/46 [00:00<?, ?it/s]

epoch roc_auc: 0.9608960960686461


In [ ]:
# score 기록하기
final_score = np.mean(score_list)

In [ ]:
#korean_col_name_list = ['clean','local','religion_model','']
#eng_model_name_list = ['clean','지역','종교','인종국적','연령','악플욕설','성소수자','성별','기타혐오','개인지칭']

#translate_dict = {}

torch.save(model.state_dict(), PATH + f'{cate}.pth')

In [ ]:
print(final_score, cate)

0.9608960960686461 성별
